In [11]:
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pandas as pd
import scipy.stats as stats
from statsmodels.regression.mixed_linear_model import MixedLM

### Question 1

**Single Factor Studies - ANOVA Model II**

Random Cell Means Model: $Y_{ij} = \mu_{i} + \epsilon_{ij}$

- $\mu_i$ are independent $N(\mu_., \sigma_\mu^2)$
- $\epsilon_{ij}$ are independent $N(0, \sigma^2)$
- $\mu_i$ and $\epsilon_{ij}$ are independent random variables
- $i = 1, 2, \dots, r$ and $j = 1, 2, \dots, n$

In [2]:
data = np.array([
    [24.4, 1, 1],
    [22.6, 1, 2],
    [23.8, 1, 3],
    [22.0, 1, 4],
    [24.5, 1, 5],
    [22.3, 1, 6],
    [25.0, 1, 7],
    [24.5, 1, 8],
    [10.2, 2, 1],
    [12.1, 2, 2],
    [10.3, 2, 3],
    [10.2, 2, 4],
    [9.9, 2, 5],
    [11.2, 2, 6],
    [12.0, 2, 7],
    [9.5, 2, 8],
    [19.2, 3, 1],
    [19.4, 3, 2],
    [19.8, 3, 3],
    [19.0, 3, 4],
    [19.6, 3, 5],
    [18.3, 3, 6],
    [20.0, 3, 7],
    [19.4, 3, 8],
    [17.4, 4, 1],
    [18.1, 4, 2],
    [16.7, 4, 3],
    [18.3, 4, 4],
    [17.6, 4, 5],
    [17.5, 4, 6],
    [18.0, 4, 7],
    [16.4, 4, 8],
    [13.4, 5, 1],
    [15.0, 5, 2],
    [14.1, 5, 3],
    [13.1, 5, 4],
    [14.9, 5, 5],
    [15.0, 5, 6],
    [13.4, 5, 7],
    [14.8, 5, 8],
    [21.3, 6, 1],
    [20.2, 6, 2],
    [20.7, 6, 3],
    [20.8, 6, 4],
    [20.1, 6, 5],
    [18.8, 6, 6],
    [21.1, 6, 7],
    [20.3, 6, 8],
], dtype=np.float32)

#### 1.a



- **Null Hypothesis ($H_0$)**: The mean sodium content is the same across all brands.<br>
Variance between the brand means $var(\sigma_\mu^2)$ = 0.

- **Alternative Hypothesis ($H_1$)**: There is a difference in the mean sodium content among the brands.<br>Variance between the brand means $var(\sigma_{\mu}^2) > 0$.

**Test Statistic**
$$ F = \frac{\text{Mean Square Between Groups (MSTR)}}{\text{Mean Square Within Groups (MSE)}} $$


**Decision Rule**
- Conclude $H_0$ if $F \leq F_{1-\alpha, r-1, r(n-1)}$
- Conclude $H_1$ if $F > F_{1-\alpha, r-1, r(n-1)}$

In [6]:
# process data into categorical variables
df = pd.DataFrame(data, columns=["Sodium", "Brand", "Replicate"])
df["Brand"] = df["Brand"].astype("category")

# setup and fit OLS model
model = ols("Sodium ~ C(Brand)", data=df).fit()

# perform ANOVA decomposition using model 2
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)
print()


# Mean Square Between Groups
dfB = anova_table.at["C(Brand)", "df"]
MSTR = anova_table.at["C(Brand)", "sum_sq"] / dfB

# Mean Square Within Groups
dfW = anova_table.at["Residual", "df"]
MSE = anova_table.at["Residual", "sum_sq"] / dfW

f_calculated = MSTR / MSE
print("Calculated F-statistic: ", f_calculated)

alpha = 0.01
f_critical = stats.f.ppf(1 - alpha, dfB, dfW)
print("Critical F-statistic: ", f_critical)

if f_calculated > f_critical:
    print("\nReject null hypothesis")
else:
    print("\nFail to reject null hypothesis")

              sum_sq    df           F        PR(>F)
C(Brand)  854.529147   5.0  238.711145  1.083748e-29
Residual   30.070003  42.0         NaN           NaN

Calculated F-statistic:  238.71114463850907
Critical F-statistic:  3.488234863858266

Reject null hypothesis


From the result of the analysis above:
- F-statistic: 238.71 > 3.48
- p-value: $1.08 \times 10^{-29} < 0.01$ 

**Conclusion**
- We reject $H_0$ and conclude $H_1$.
- There is significant evidence to suggest that the mean sodium content is not the same across all brands in the dataset.

#### 1.b

In [19]:
alpha = 0.01
overall_mean = df["Sodium"].mean()
n_total = len(df)
standard_error = np.sqrt(MSE)

# calculate confidence intervals
t_value = stats.t.ppf(1 - alpha / 2, dfW)

confidence_interval = (
    overall_mean - t_value * standard_error / np.sqrt(n_total),
    overall_mean + t_value * standard_error / np.sqrt(n_total),
)

print("Overall mean: ", overall_mean)
print("Standard error: ", standard_error)
print()
print("Confidence interval: ", confidence_interval)

Overall mean:  17.629168
Standard error:  0.8461397355860263

Confidence interval:  (17.29965340518216, 17.95868170834323)


Mean Sodium content across all brands are within the interval (17.3 - 17.96).

#### 1.c

In [18]:
# observations per group
k = df.groupby("Brand", observed=False).size().min()

# variance between group means (sigma^2_mu)
numerator = (MSTR - MSE) / k
denominator = numerator + MSE

variance_proportion = numerator / denominator
print("Variance proportion: ", variance_proportion)

Variance proportion:  0.9674414442545144


Variance proportion $\approx 0.967$
<br>
96.7% of the total variability in the sodium content is due to the variability between the brands.

#### 1.d

In [17]:
# fitting the model using Maximum Likelihood Estimation
ml_model = MixedLM.from_formula("Sodium ~ 1", groups="Brand", re_formula="1", data=df)
ml_result = ml_model.fit(method="lbfgs")

# extract variance components
ml_var_components = ml_result.cov_re.iloc[0, 0]
ml_error = ml_result.scale

# calculate variance proportion
ml_quantity = ml_var_components / (ml_var_components + ml_error)
print("Estimated variance proportion (ML): ", ml_quantity)

Estimated variance proportion (ML):  0.9674404956166446


In [16]:
# fitting the model using Restricted Maximum Likelihood Estimation
df["Replicate"] = df.groupby("Brand", observed=False).cumcount() + 1

reml_model = MixedLM.from_formula("Sodium ~ 1", groups="Brand", re_formula="1", data=df)
reml_result = reml_model.fit(reml=True, method="lbfgs")

# extract variance components
reml_var_components = reml_result.cov_re.iloc[0, 0]
reml_error = reml_result.scale

# calculate variance proportion
reml_quantity = reml_var_components / (reml_var_components + reml_error)
print("Estimated variance proportion (REML): ", reml_quantity)

Estimated variance proportion (REML):  0.9674404956166446


**Results**:
- ANOVA: 96.7%
- MLE: 96.7%
- REML: 96.7%

All three methods generated the same result.

To include the ANOVA results in this comparison, let's recap and compare the findings from the three different methods: ANOVA, Maximum Likelihood (ML), and Restricted Maximum Likelihood (REML). We focused on estimating the quantity \(\sigma^2_{\mu}/(\sigma^2_{\mu} + \sigma^2)\), which reflects the proportion of total variance attributed to the differences between group means.

### Results Comparison
1. **ANOVA Method**: Estimated this quantity as approximately 0.967.
2. **ML Method**: Also estimated the quantity as approximately 0.967.
3. **REML Method**: Yielded the same estimate, approximately 0.967.

### Analysis and Preference

- **Consistency of Results**: All three methods yielded the same estimate in this case. This consistency suggests that, for your dataset, the specific choice of method might not significantly impact the estimate of the variance components.
- **When to Prefer Each Method**:
  - **ANOVA**: Best suited for simpler models where the focus is on comparing group means and assuming fixed effects. However, it does not account for the potential bias in estimating variance components, especially in models with a large number of parameters or small sample sizes.
  - **ML**: Useful when comparing different fixed-effects models, as it allows for the comparison of likelihoods across models. However, it can be biased in estimating variance components, particularly with smaller sample sizes.
  - **REML**: Provides less biased estimates of variance components, especially beneficial for small sample sizes or complex models. It's generally the preferred method for variance component estimation in mixed models.

### Conclusion
- **In Your Case**: Given that all three methods produced identical results for your dataset, it suggests that your data are robust enough that the potential biases of ML are not a significant factor. In such cases, any of the methods could be suitable.
- **General Preference**: In more complex scenarios or with smaller sample sizes, REML is usually preferred due to its reduced bias in estimating variance components. However, if you're interested in comparing models with different fixed effects, ML would be more appropriate.
- **Overall**: Considering the nature of most practical applications, where either sample sizes are not exceedingly large, or models tend to be more complex, REML is generally preferred for its accuracy in estimating variance components in mixed models. But for your specific dataset and analysis, any method seems to be equally valid as they all yielded the same result.